In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df1 = pd.read_csv('dftRoadSafety_Accidents_2016.csv')
df2 = pd.read_csv('Cas.csv')
df3 = pd.read_csv('MakeModel2016.csv')

pd.set_option('display.max_columns', None)

# Merging the Datasets

In [ ]:
df = pd.merge(df1, df2, on=['Accident_Index'])
df.tail()

In [ ]:
df_merged = pd.merge(df, df3, on=['Accident_Index'])
df_merged.head()

In [ ]:
#Create a datetime column
df_merged['Datetime'] = df_merged['Date'] + ' ' + df_merged['Time']  + ':00'
df_merged['Datetime'] = pd.to_datetime(df_merged['Datetime'])

In [ ]:
from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh.models import HoverTool, ColumnDataSource, Slider, Select
from bokeh.tile_providers import get_provider, Vendors
from bokeh.layouts import row, column
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
output_notebook()

In [ ]:
hotspots = df_merged.groupby(by=['Longitude','Latitude','Datetime']).mean()[['Number_of_Casualties','Weather_Conditions']]
hotspots.Weather_Conditions = hotspots.Weather_Conditions.astype(int)
hotspots['circle_sizes'] = hotspots['Number_of_Casualties'] * 20 / hotspots['Number_of_Casualties'].max()
hotspots.reset_index(inplace=True)
hotspots['time'] = hotspots.Datetime.dt.hour

In [ ]:
def wgs84_to_web_mercator(df, lon="Longitude", lat="Latitude"):
    '''
    Function to transform coordinates from WGS84 system to Mercator coordinates
    '''

    k = 6378137
    df["LON"] = df[lon] * (k * np.pi/180.0)
    df["LAT"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k

    return df

In [ ]:
# Tell Bokeh to use 'hotspots' as the source of the data
data_source = ColumnDataSource(data=wgs84_to_web_mercator(hotspots))

# List navigation tools
TOOLS = 'pan,wheel_zoom,zoom_in,zoom_out,box_zoom,reset,save'
# Create figure
p = figure(title = 'Road Accidents hotspots location in the United Kingdon in 2016', tools = TOOLS)

# instatiate the tile source provider to use for the map
tile_provider = get_provider(Vendors.OSM)
# add the back ground basemap
p.add_tile(tile_provider)

# Add accident points using coordinates ('Longitude' & 'Latitude')
p.circle(x='LON',
         y='LAT', 
         size='circle_sizes',
         line_color="#FF0000", 
         fill_color="#FF0000",
         fill_alpha=0.05, 
         source=data_source)

# Set tooltips that will appear when the user hovers over a data point on our map. 
p.add_tools(HoverTool( tooltips=[
    ("No. of Casualties", "@Number_of_Casualties"),
    ("(Long, Lat)", "(@Longitude, @Latitude)"),
    
]))

show(p)

In [ ]:
Weather_Conditions = {'Fine no high winds' : 1, 'Raining no high winds' : 2, 'Snowing no high winds' : 3, 
                      'Fine + high winds' : 4, 'Raining + high winds' : 5, 'Snowing + high winds' : 6, 
                      'Fog or mist' : 7, 'Other' : 8, 'Unknown' : 9}
w = [i for i in Weather_Conditions.keys()]

defaultdata = hotspots[(hotspots.time == 12) & (hotspots.Weather_Conditions == 1)]
defaultdata['circle_sizes'] = defaultdata['Number_of_Casualties'] * 20 / defaultdata['Number_of_Casualties'].max()

data_source = ColumnDataSource(data=wgs84_to_web_mercator(defaultdata))

output_file("uk_road_accidents.html")
# List navigation tools
TOOLS = 'pan,wheel_zoom,zoom_in,zoom_out,box_zoom,reset,save'
# Create figure
p = figure(x_axis_type="mercator", y_axis_type="mercator", 
           title = 'Spatial distribution of accident hotspots in the United Kingdon in 2016', tools = TOOLS
          )

# Select which tiles to use for the map
tile_provider = get_provider(Vendors.OSM)
p.add_tile(tile_provider) # Add map tile

# Add accident points using coordinates ('Longitude' & 'Latitude')
p.circle(x='LON',
         y='LAT', 
         size='circle_sizes',
         line_color="#FF0000", 
         fill_color="#FF0000",
         fill_alpha=0.05, 
         source=data_source)

# Set tooltips that will appear when the user hovers over a data point on our map. 
p.add_tools(HoverTool( tooltips=[
    ("No. of Casualties", "@Number_of_Casualties"),
    ("(Long, Lat)", "(@Longitude, @Latitude)"),
]))

def update_data(hr, w, s):
    data = hotspots[(hotspots.time == hr) & (hotspots.Weather_Conditions == w)]
    data['circle_sizes'] = data['Number_of_Casualties'] * s / data['Number_of_Casualties'].max()
    return data

def update_plot(attr, old, new):
    '''
    callback function to receive data from the widget and update plot
    '''
    hr = timer.value
    w = Weather_Conditions[weather.value]
    s = size.value

    new_data =  update_data(hr, w, s)
    data_source.data = new_data
    
# Set up widgets
timer = Slider(title='Time of the day',start=0,end=23,step=1,value=12)
weather = Select(title="Weather Condition:", value="Fine no high winds", options=w)
size = Slider(title="Zoom accident points", start=10, end=40, step=2, value=20)


# Set up layouts and add to document
layout = row(p, column(timer, weather, size, width=250))

def modify_doc(doc):
    doc.add_root(row(layout, width=800))
    #doc.title = "Sliders"
    for widgets in [timer, weather, size]:
        widgets.on_change('value', update_plot)

handler = FunctionHandler(modify_doc)
app = Application(handler)
#save(p)
show(app)